In [1]:
import pydub # faster than librosa
import numpy      as np
import pandas     as pd
import time

from coughvid.pytorch.coughvid_dataset import CoughvidDataset
from coughvid.pytorch.coughvid_dataset import CoughvidDataset
from coughvid.pytorch.coswara_dataset import CoswaraDataset
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.models import resnet50, resnet18

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy

import wandb
#from torch.utils.tensorboard import SummaryWriter
#writer = SummaryWriter()

In [2]:
dir = 'C:/COUGHVID_public_dataset/public_dataset/'

full_dataset = CoughvidDataset(dir, 'metadata_compiled.csv', get_features=True)

dataframe = full_dataset.dataframe

minority_class_count = len(dataframe[dataframe['status']==1])

print(f'{minority_class_count} samples in the minority class.')

sample_dataset = CoughvidDataset(dir, 'metadata_compiled.csv', get_features=True, samples_per_class=minority_class_count)

Mask file C:/COUGHVID_public_dataset/public_dataset/ does not exist. Calculating masks on the fly.
8997 records ready to load across 2 groups.
699 samples in the minority class.
Mask file C:/COUGHVID_public_dataset/public_dataset/ does not exist. Calculating masks on the fly.
1398 records ready to load across 2 groups.


In [2]:
dir = './data/coswara/'

full_dataset = CoswaraDataset(dir, 'filtered_data.csv', get_features=True)

dataframe = full_dataset.dataframe

minority_class_count = len(dataframe[dataframe['covid_status']==1])

print(f'{minority_class_count} samples in the minority class.')

sample_dataset = CoswaraDataset(dir, 'filtered_data.csv', get_features=True, samples_per_class=minority_class_count)

1731 records ready to load across 2 groups.
378 samples in the minority class.
756 records ready to load across 2 groups.


In [3]:
# split data into training and test samples
train_indices, test_indices = train_test_split(np.arange(0,len(sample_dataset)-1), test_size=0.25)

batch_size  = 1
num_workers = 2

train_loader  = DataLoader(sample_dataset, 
                           num_workers=num_workers,
                           sampler=SubsetRandomSampler(train_indices)
                          )

test_loader   = DataLoader(sample_dataset, 
                           num_workers=num_workers,
                           sampler=SubsetRandomSampler(test_indices)
                          )

dataloaders = {
    "train": train_loader,
    "test": test_loader
}

In [4]:
# load model and change output shape for binary prediction
model = resnet18()

model.fc = torch.nn.Sequential(
    torch.nn.Dropout(0.10,inplace=True),
    torch.nn.Linear(
        in_features=512, #2048 for resnet50
        out_features=1
    ),
    torch.nn.Sigmoid()
)

model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=True)


optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.BCELoss()

model.double()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2):

In [7]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs
    
batch_size = 100
n_iters = 3000
EPOCHS = int(n_iters / (len(sample_dataset) / batch_size))

lr_rate = 0.00001    
    
model = LogisticRegression(4050,1)
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)
criterion = torch.nn.BCELoss()
model.double()


LogisticRegression(
  (linear): Linear(in_features=4050, out_features=1, bias=True)
)

In [6]:
'''RESNET training code adapted from https://www.kaggle.com/gxkok21/resnet50-with-pytorch'''

if torch.cuda.is_available():
    model = model.cuda()
    print('Using GPU.')
    
wandb.init()    
wandb.watch(model)

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

EPOCHS = 50

for i in range(EPOCHS):
    for phase in ["train", "test"]:
        if phase == "train":
            model.train()
        else:
            model.eval()
        
        samples = 0
        loss_sum = 0
        correct_sum = 0
        for j, batch in enumerate(dataloaders[phase]):
            X,labels = batch
            #print(labels)
            #print(len(X),len(X[0]),len(X[1]))
            #print(labels[None,...].double())
            #X = torch.Tensor(X)
            #print(X.shape)
            #labels = 1-labels
            if torch.cuda.is_available():
                X = X.cuda()
                labels = labels.cuda()
                #model = model.cuda()

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                y = model(X[None,...].double())
                #y = model(X.flatten().double())
                #print(y,labels)
                loss = criterion(
                    y,
                    #torch.clip(y,0,1), 
                    labels[None,...].double()
                    #labels.double()
                )
                #print(loss.item())
                #print(labels[None,...].double())
                
                #writer.add_scalar("Loss/train", loss, i)

                if phase == "train":
                    loss.backward()
                    optimizer.step()
                    
                loss_sum += loss.item() * X.shape[0] # We need to multiple by batch size as loss is the mean loss of the samples in the batch
                samples += X.shape[0]
                num_corrects = torch.sum((y >= 0.5).float() == labels[0].float())
                correct_sum += num_corrects
                
                # Print batch statistics every 50 batches
                if j % 50 == 49 and phase == "train":
                    #wandb.log({'Train Loss': float(loss_sum) / float(samples), 'Train Accuracy': float(correct_sum) / float(samples)})

                    print("{}:{} - loss: {}, acc: {}".format(
                        i + 1, 
                        j + 1, 
                        float(loss_sum) / float(samples), 
                        float(correct_sum) / float(samples)
                    ))
                    
                
        # Print epoch statistics
        epoch_acc = float(correct_sum) / float(samples)
        epoch_loss = float(loss_sum) / float(samples)
        print("epoch: {} - {} loss: {}, {} acc: {}".format(i + 1, phase, epoch_loss, phase, epoch_acc))
        
        # Deep copy the model
        if phase == "test" and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "resnet18_coswara2.pth")

Using GPU.


wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Filip/.netrc


1:50 - loss: 0.8624618969590682, acc: 0.48
1:100 - loss: 0.8373175373250368, acc: 0.51
1:150 - loss: 0.7727786192988627, acc: 0.5866666666666667
1:200 - loss: 0.7546751775830042, acc: 0.61
1:250 - loss: 0.7561352259109202, acc: 0.592
1:300 - loss: 0.7497283396721769, acc: 0.5833333333333334
1:350 - loss: 0.7458537453787911, acc: 0.58
1:400 - loss: 0.7447015140363002, acc: 0.575
1:450 - loss: 0.7413862589910749, acc: 0.5755555555555556
1:500 - loss: 0.7436514174517306, acc: 0.568
1:550 - loss: 0.7357599998003134, acc: 0.5672727272727273
epoch: 1 - train loss: 0.7389071931930815, train acc: 0.5653710247349824
epoch: 1 - test loss: 0.698726289652009, test acc: 0.5396825396825397
2:50 - loss: 0.7100536460617322, acc: 0.62
2:100 - loss: 0.6979814334310923, acc: 0.59
2:150 - loss: 0.7031320844274089, acc: 0.5666666666666667
2:200 - loss: 0.7151235594452889, acc: 0.555
2:250 - loss: 0.7172521625923268, acc: 0.568
2:300 - loss: 0.7118804708077459, acc: 0.5733333333333334
2:350 - loss: 0.720377

...

47:450 - loss: 0.015195056090219585, acc: 0.9888888888888889
47:500 - loss: 0.021132529652216098, acc: 0.986
47:550 - loss: 0.02284699402236588, acc: 0.9854545454545455
epoch: 47 - train loss: 0.02221770014573934, train acc: 0.9858657243816255
epoch: 47 - test loss: 0.9736134101871254, test acc: 0.5925925925925926
48:50 - loss: 0.0005312403178785842, acc: 1.0
48:100 - loss: 0.017554080465044912, acc: 0.99
48:150 - loss: 0.02141813491385966, acc: 0.9866666666666667
48:200 - loss: 0.02782613374945586, acc: 0.985
48:250 - loss: 0.022521000186786086, acc: 0.988
48:300 - loss: 0.01881529949479953, acc: 0.99
48:350 - loss: 0.016162941568849485, acc: 0.9914285714285714
48:400 - loss: 0.014168408141160178, acc: 0.9925
48:450 - loss: 0.013780737668645416, acc: 0.9933333333333333
48:500 - loss: 0.019581519704044238, acc: 0.99
48:550 - loss: 0.01803910780663822, acc: 0.990909090909091
epoch: 48 - train loss: 0.01753186536181098, train acc: 0.991166077738516
epoch: 48 - test loss: 1.30431130815657

59:100 - loss: 0.02497564619311543, acc: 1.0
59:150 - loss: 0.03363956270306649, acc: 0.9866666666666667
59:200 - loss: 0.12836455365763436, acc: 0.965
59:250 - loss: 0.10991769969049883, acc: 0.968
59:300 - loss: 0.10020432284828262, acc: 0.97
59:350 - loss: 0.09097771681648704, acc: 0.9742857142857143
59:400 - loss: 0.08510611278915964, acc: 0.975
59:450 - loss: 0.07767803722935582, acc: 0.9777777777777777
59:500 - loss: 0.07125799552013574, acc: 0.98
59:550 - loss: 0.06938488226329663, acc: 0.98
epoch: 59 - train loss: 0.06851129507618256, train acc: 0.980565371024735
epoch: 59 - test loss: 1.0000642805368838, test acc: 0.5661375661375662
60:50 - loss: 0.024338321139015778, acc: 0.98
60:100 - loss: 0.04877823226194966, acc: 0.96
60:150 - loss: 0.03957108397248469, acc: 0.9666666666666667
60:200 - loss: 0.032486940013374876, acc: 0.975
60:250 - loss: 0.02681491111727964, acc: 0.98
60:300 - loss: 0.024376285873888584, acc: 0.9833333333333333
60:350 - loss: 0.05568526259587778, acc: 0.

70:450 - loss: 0.02328813193431343, acc: 0.9822222222222222
70:500 - loss: 0.020991745214650605, acc: 0.984
70:550 - loss: 0.01912051076109265, acc: 0.9854545454545455
epoch: 70 - train loss: 0.01859584514422063, train acc: 0.9858657243816255
epoch: 70 - test loss: 0.907350183754939, test acc: 0.5396825396825397
71:50 - loss: 0.009427242450918454, acc: 1.0
71:100 - loss: 0.027439489180171292, acc: 0.99
71:150 - loss: 0.020516613097789453, acc: 0.9933333333333333
71:200 - loss: 0.023013220908125267, acc: 0.99
71:250 - loss: 0.020719495833281764, acc: 0.992
71:300 - loss: 0.0218303599422496, acc: 0.99
71:350 - loss: 0.02225414154386261, acc: 0.9885714285714285
71:400 - loss: 0.026717850479795427, acc: 0.9825
71:450 - loss: 0.02377643646387639, acc: 0.9844444444444445
71:500 - loss: 0.021424009275871594, acc: 0.986
71:550 - loss: 0.0195035509219851, acc: 0.9872727272727273
epoch: 71 - train loss: 0.018960818855617796, train acc: 0.9876325088339223
epoch: 71 - test loss: 0.9577062965408104

epoch: 81 - test loss: 1.2666413620598493, test acc: 0.5767195767195767
82:50 - loss: 0.00014077801344586992, acc: 1.0
82:100 - loss: 0.013005024428769187, acc: 0.99
82:150 - loss: 0.014325047857760126, acc: 0.9866666666666667
82:200 - loss: 0.02069232075465455, acc: 0.985
82:250 - loss: 0.025808560860684348, acc: 0.98
82:300 - loss: 0.02151985341907148, acc: 0.9833333333333333
82:350 - loss: 0.01845961291768809, acc: 0.9857142857142858
82:400 - loss: 0.018482396804129168, acc: 0.985
82:450 - loss: 0.016434618430048658, acc: 0.9866666666666667
82:500 - loss: 0.01955031591996774, acc: 0.986
82:550 - loss: 0.017776663040092656, acc: 0.9872727272727273
epoch: 82 - train loss: 0.017276012502176866, train acc: 0.9876325088339223
epoch: 82 - test loss: 1.215654469297015, test acc: 0.5873015873015873
83:50 - loss: 3.793878695027998e-05, acc: 1.0
83:100 - loss: 0.007611893285349131, acc: 0.99
83:150 - loss: 0.014981821730468185, acc: 0.9866666666666667
83:200 - loss: 0.015094498319724394, acc:

93:250 - loss: 0.01904089626632267, acc: 0.992
93:300 - loss: 0.015892647479993232, acc: 0.9933333333333333
93:350 - loss: 0.01768654962692598, acc: 0.9914285714285714
93:400 - loss: 0.01550486358883768, acc: 0.9925
93:450 - loss: 0.017073550929673463, acc: 0.9911111111111112
93:500 - loss: 0.01635662667461336, acc: 0.992
93:550 - loss: 0.014884291858033026, acc: 0.9927272727272727
epoch: 93 - train loss: 0.014486105380615878, train acc: 0.9929328621908127
epoch: 93 - test loss: 1.6233988993629715, test acc: 0.5238095238095238
94:50 - loss: 0.025370584171785525, acc: 0.98
94:100 - loss: 0.028951916977885093, acc: 0.97
94:150 - loss: 0.019343679145983936, acc: 0.98
94:200 - loss: 0.01898474259954368, acc: 0.98
94:250 - loss: 0.019251248176540572, acc: 0.98
94:300 - loss: 0.018379621262326548, acc: 0.9833333333333333
94:350 - loss: 0.02098354224349264, acc: 0.9828571428571429
94:400 - loss: 0.019604258488703813, acc: 0.985
94:450 - loss: 0.017436153291453288, acc: 0.9866666666666667
94:5